In [ ]:

import os
import shutil
import pandas as pd
import numpy as np
from itertools import chain
import matplotlib.pyplot as plt
from PIL import Image, ImageStat


import os


In [ ]:
#Reading data entries
df = pd.read_csv("/kaggle/input/data/Data_Entry_2017.csv")
df.head()



In [ ]:
#Extract full paths using glob
from glob import glob
my_glob = glob('/kaggle/input/data/images_*/images/*.png')
print('Number of Observations: ', len(my_glob))
img_paths  = {os.path.basename(x): x for x in my_glob}
#Add paths to df and remove Image Index column since it's not necessary with paths in hand
df['Image index'] = df['Image Index'].map(img_paths.get)
df.drop(['Image Index'], axis=1,inplace = True)
df.head()


In [ ]:
# Get the unqiue labels
classes_split = pd.DataFrame({'Finding Labels': chain.from_iterable(df['Finding Labels'].str.split('|'))})
labels = classes_split['Finding Labels'].unique()
labels

In [ ]:
# Encoding dataframe
encoded_df = pd.DataFrame(columns=labels)
encoded_df.insert(0, 'Image index', df['Image index'])
for col in encoded_df.columns:
    if col != 'Image index':
        encoded_df[col] = 0

for i, row in df.iterrows():
    values = row['Finding Labels'].split('|')
    for val in values:
        encoded_df.iloc[i, encoded_df.columns.get_loc(val)] = 1 

In [ ]:
def distribution(encoded_df):
    samples = {}
    for col in encoded_df.columns:
        if col != 'Image index':
            tot = encoded_df[col].sum()
            samples[col] = tot
            print(col, end=" ") 
            print(tot)

    # Current split of data between the different classes
    plt.figure(figsize=(25,5))
    plt.bar(range(len(samples)), list(samples.values()), align='center')
    plt.xticks(range(len(samples)), list(samples.keys()))
    plt.show()
distribution(encoded_df)

In [ ]:
# Drop all 'No Finding' data
encoded_df = encoded_df[encoded_df['No Finding'] == 0]
encoded_df = encoded_df.drop(columns='No Finding')


In [ ]:
distribution(encoded_df)

In [ ]:
def get_Full_path(df):
    my_glob = glob('/kaggle/input/data/images_*/images/*.png')
   
    img_paths = {os.path.basename(x): x for x in my_glob}
    #Add paths to df and remove Image Index column since it's not necessary with paths in hand
    df['Image index'] = df['Image Index'].map(img_paths.get)
    df.drop(['Image Index'], axis=1,inplace = True)
    df.head()



In [ ]:
# Organize the data into training and testing folders given the list of files
train_val_list = pd.read_csv("../input/data/train_val_list.txt",header = None)
train_val_list.columns = ['Image Index']
get_Full_path(train_val_list)

test_list = pd.read_csv("../input/data/test_list.txt",header = None)
test_list.columns = ['Image Index']
get_Full_path(test_list)
train_val_list = pd.merge(train_val_list, encoded_df , on='Image index', how='inner')

test_list = pd.merge(test_list, encoded_df , on='Image index', how='inner')

In [ ]:
train_val_list.to_csv('train_val_list.csv', index=False)

In [ ]:
np.random.rand(len(train_val_list)) 

In [ ]:
msk = np.random.rand(len(train_val_list)) < 0.90
train_df = train_val_list[msk]
val_df = train_val_list[~msk]

In [ ]:
msk = np.random.rand(len(test_list)) > 0.80
new_test = test_list[msk]
new_train = pd.concat([train_df, test_list[~msk]])

In [ ]:
new_test.head()

In [ ]:
new_train.to_csv('train_list.csv', index=False)
val_df.to_csv('val_list.csv', index=False)
new_test.to_csv('test_list.csv', index=False)
encoded_df.to_csv('Encoded_Multilabel.csv', index=False)

In [ ]:
test_df=pd.read_csv('../input/data-labels/test_list.csv')
train_df=pd.read_csv('../input/data-labels/train_list.csv')
val_df=pd.read_csv('../input/data-labels/val_list.csv')


In [ ]:
test_df.head()

In [ ]:
train_df.head()

In [ ]:
val_df.head()